# Modelproject: IS-LM Model

For this project, we will utilize a simple IS-LM model for a closed economy. The project is divided into three parts:

**Building the Model:** We will construct the IS-LM model by using substitution and solving functions to derive the IS and LM curves, and then find the equilibrium.

**Parameter Analysis:** In this part, we will assign different parameter values chosen at random to illustrate the curves and equilibrium on graphs. This will allow us to analyze how fiscal and monetary shocks impact the equilibrium.

**Exploring Open Economy Effects:** Finally, we will investigate how opening up the economy influences the equilibrium values.

## Description and model build-up

The model consists of the following equations:

1. **Planned expenditures on goods and services:** $$PE = C + I + \bar{G}$$
2. **Private consumption of goods:** $$C = a + b(Y-\bar{T})$$
3. **Equilibrium in the goods market:** $$Y = PE$$
4. **Investment demand:** $$I = c - d r$$
5. **Equilibrium condition in the money market:** $$\frac{\bar{M}}{\bar{P}} = L$$
6. **Real demand for money:** $$L = e Y - f r$$
7. **Production function:** $$Y = g N$$

Where $PE$ is the planned expenditures on goods and services, $C$ is private demand for consumption, $I$ is investment demand, $\bar{G}$ is public demand for consumption, $\bar{T}$ is a lumpsum tax, $r$ is the real interest rate, $\bar{M}$ is the nominaal money supply, $\bar{P}$ is the consumer price index, $L$ is the real demand for money and $N$ is the workforce. The parameters $a, b, c, d, e, f$ and $g$ are all positive. Further, $b$ is less than 1. A bar over a variable means, that the variable is exogenous. 

Import and set magics

In [97]:
# Import necessary libraries
import pandas as pd  # Import pandas for data manipulation
import matplotlib.pyplot as plt  # Import matplotlib for plotting
import numpy as np  # Import numpy for numerical computing
from scipy import optimize
from scipy.optimize import fsolve
import sympy as sp
from IPython.display import display

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


This code performs the following steps:

**Import Necessary Modules**:
   - The `ISLM_alg` class is imported from the `ISLM_new` Python file.
   - The `sympy` library is imported for symbolic mathematics.

**Initialize the Model**:
   - An instance of the `ISLM_alg` class is created and assigned to the variable `model`.

**Derive IS and LM Curve Solutions**:
   - The `derive_IS` method is called to derive solutions for the IS curve.
   - The `derive_LM` method is called to derive solutions for the LM curve.
   - The solutions for the output (Y) and the interest rate (r) are stored in `IS_solution_Y`, `IS_solution_r`, `LM_solution_Y`, and `LM_solution_r`.

**Display IS and LM Curve Solutions**:
   - The solutions for \( Y \) and \( r \) from the IS and LM curves are displayed.


In [98]:
# Import the class from the Python file
from ISLM_new import ISLM_alg

# Initialize the ISLM_alg
model = ISLM_alg()

# Derive the IS curve solutions
IS_solution_Y, IS_solution_r = model.derive_IS()

# Derive the LM curve solutions
LM_solution_Y, LM_solution_r = model.derive_LM()

# Display the IS curve solutions for Y and r
print("IS Curve Solution (Y):")
if IS_solution_Y:
    for sol in IS_solution_Y:
        display(sp.Eq(model.Y, sol))

print("IS Curve Solution (r):")
if IS_solution_r:
    for sol in IS_solution_r:
        display(sp.Eq(model.r, sol))

# Display the LM curve solutions for Y and r
print("LM Curve Solution (Y):")
if LM_solution_Y:
    for sol in LM_solution_Y:
        display(sp.Eq(model.Y, sol))


print("LM Curve Solution (r):")
if LM_solution_r:
    for sol in LM_solution_r:
        display(sp.Eq(model.r, sol))

IS Curve Solution (Y):


Eq(Y, (-G + T*b - a - c + d*r)/(b - 1))

IS Curve Solution (r):


Eq(r, (G - T*b + Y*b - Y + a + c)/d)

LM Curve Solution (Y):


Eq(Y, (M + P*f*r)/(P*e))

LM Curve Solution (r):


Eq(r, (-M + P*Y*e)/(P*f))

**Find and display the equilibrium**:
   - The find_equilibrium method is called to solve the IS and LM equations simultaneously, finding the values of \( Y \) and \( r \) that satisfy both equations.
   - The equilibrium solutions for \( Y \) and \( r \) are displayed, showing where the IS and LM curves intersect.


In [123]:

# Find the equilibrium for Y and r
equilibrium = model.find_equilibrium()

# Display the equilibrium solutions
print("Equilibrium where IS curve Y equals LM curve Y for r:")
if equilibrium:
    for sol in equilibrium:
        if model.r in sol:
            display(sp.Eq(model.r, sol[model.r]))
        if model.Y in sol:
            display(sp.Eq(model.Y, sol[model.Y]))
else:
    print("No equilibrium found.")


Equilibrium where IS curve Y equals LM curve Y for r:


Eq(r, (-G*P*e - M*b + M + P*T*b*e - P*a*e - P*c*e)/(P*b*f - P*d*e - P*f))

Eq(Y, (-G*P*f - M*d + P*T*b*f - P*a*f - P*c*f)/(P*b*f - P*d*e - P*f))

## Analysis of IS-LM model

In [124]:
# Initialize the model with adjusted parameters for better curve fitting
model = ISLMclass(
    a=2, b=0.6, c=2, d=0.1, e=6, f=0.2, g=1.5,
    T=1, G=1, M=5, P=2, r=0.04, N=500
)

# Define functions to compute IS and LM interest rates
def IS_r(Y):
    # Adjusting to return r directly
    return (1 / model.par.d) * (model.par.a + model.par.c - model.par.b * model.par.T + model.par.G - (1 - model.par.b) * Y)

def LM_r(Y):
    # Adjusting to return r directly
    return (model.par.e / model.par.f) * Y - (1 / model.par.f) * (model.par.M / model.par.P)

# Generate Y values and compute r for IS and LM curves
Y_values = np.linspace(50, 4000, 400)  # Adjusted range if needed
IS_r_values = [IS_r(Y) for Y in Y_values]
LM_r_values = [LM_r(Y) for Y in Y_values]

# Find equilibrium by minimizing the difference between the IS and LM r values
Y_eq_guess = 500
Y_eq = fsolve(lambda Y: abs(IS_r(Y) - LM_r(Y)), Y_eq_guess)
r_eq_IS = IS_r(Y_eq[0])
r_eq_LM = LM_r(Y_eq[0])

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(Y_values, IS_r_values, label='IS Curve', color='blue')
plt.plot(Y_values, LM_r_values, label='LM Curve', color='red')
plt.scatter(Y_eq, r_eq_IS, color='green', s=100, label=f'Equilibrium (Y={Y_eq[0]:.2f}, r={r_eq_IS:.4f})')
plt.title('IS-LM Model')
plt.xlabel('Output, Y')
plt.ylabel('Interest Rate, r')
plt.legend()
plt.grid(True)
plt.show()

NameError: name 'ISLMclass' is not defined